In [1]:
import sys
import os
import time
import numpy as np
from scipy.sparse import coo_matrix
sys.path.insert(0, '/mnt/cbis/home/amandakau/pyReCoDe')
from pyrecode.recode_reader import ReCoDeReader
from pyrecode.utils.converters import l1_to_l4_converter
from IPython.display import clear_output

In [2]:
class ReaderNode:
    def __init__(self, file_name, file_id):
        self._file_name = file_name
        self._file_id = file_id
        self._is_read = False
        
        self._reader = ReCoDeReader(file_name, is_intermediate=True)
        self._reader.open(print_header=False)
        header = self._reader.get_header().as_dict()
        
        self._shape = (header['nx'], header['ny'])
        self._curr_position = self._reader._current_frame_index
        
    def get_next_batch(self, batch_size=60):
        count = 0
        batch_data = []
        
        while count < batch_size and self._is_read == False:
            frame_data = self._reader.get_next_frame()
            first_try = True

            if frame_data is None:
                # Failed to load more data after waiting, part file is probably done updating
                if first_try == False:
                    print(f"Reached end of part file {file_id}.")
                    self.is_read = True
                    break
                
                # Else, wait for part file to update. Usually part files will be updated once in 1-2 mins.
                else:
                    print(f"Waiting for part file {file_id} to update.")
                    first_try = False
                    time.sleep(240)
            else:
                batch_data.append(frame_data)
                count += 1
            del(frame_data)
        
        self._curr_position = self._reader._current_frame_index
        return batch_data
    
    def get_shape(self):
        return self._shape
    
    def get_status(self):
        read_status = 'read' if self._is_read else 'still being read'
        pos_status = '' if self._is_read else f' (Current position at {self._curr_position})'
        print(f'{self._file_name} corresponding to part file #{self._file_id} is {read_status}{pos_status}')
    
    def close(self):
        self._reader.close()

In [3]:
class Converter:
    def __init__(self, folder_path, file_name, num_parts, off_part):
        self._folder_path = folder_path
        self._file_name = file_name
        self._num_parts = num_parts
        self._off_part = off_part
        self._part_files = {}
        
    def start(self):
        print("Starting converter.")
        for index in range(self._off_part, self._off_part+self._num_parts):
            part_file_name = os.path.join(self._folder_path, self._file_name + '_part' + '{0:03d}'.format(index))
            self._part_files[index] = ReaderNode(part_file_name, index)
        print(self._part_files)
    
    def convert(self, part_file_id, num_frames=1):
        data = self._part_files[part_file_id].get_next_batch(batch_size=num_frames)
        shape = self._part_files[part_file_id].get_shape()
        
        l4_data = {}
        for frame in data:
            frame_id = list(frame.keys())[0]
            l4_frame = l1_to_l4_converter(frame, shape, area_threshold=1)[frame_id]['data']
            l4_data[frame_id] = {}
            l4_data[frame_id]['data'] = coo_matrix(l4_frame, dtype=l4_frame.dtype)
        
        return l4_data
    
    def close(self):
        for index in range(self._off_part, self._off_part+self._num_parts):
            self._part_files[index].close()
        print("Converter closed.")

In [4]:
_data_folder = '/scratch/loh/deepan/12Mar21/captures/'
_tag = 'streampix_2k_40fps_run_overnight'
_num_part_files = 1

converter = Converter(_data_folder, _tag + '.rc1', _num_part_files, 2)
converter.start()

Starting converter.
{2: <__main__.ReaderNode object at 0x7f0420c842b0>}


In [5]:
it = 0
is_ended = False
rc4_data = {}

# Try for 1000 iterations (100,000 frames)
while not is_ended:
    if it == 1000:
        is_ended = True
        break
        
    converted_data = converter.convert(2, num_frames=100)
    clear_output(wait=True)
    
    if converted_data == None:
        is_ended = True
        break
    else:
        rc4_data.update(converted_data)
    
    it += 1
    if len(rc4_data) > 500:
        rc4_data.clear()
        
converter.close()

Converter closed.


In [7]:
rc4_data

{961415: {'data': <2048x2048 sparse matrix of type '<class 'numpy.bool_'>'
  	with 17576 stored elements in COOrdinate format>},
 961416: {'data': <2048x2048 sparse matrix of type '<class 'numpy.bool_'>'
  	with 17423 stored elements in COOrdinate format>},
 961599: {'data': <2048x2048 sparse matrix of type '<class 'numpy.bool_'>'
  	with 17712 stored elements in COOrdinate format>},
 961600: {'data': <2048x2048 sparse matrix of type '<class 'numpy.bool_'>'
  	with 17403 stored elements in COOrdinate format>},
 961601: {'data': <2048x2048 sparse matrix of type '<class 'numpy.bool_'>'
  	with 17370 stored elements in COOrdinate format>},
 961602: {'data': <2048x2048 sparse matrix of type '<class 'numpy.bool_'>'
  	with 17577 stored elements in COOrdinate format>},
 961603: {'data': <2048x2048 sparse matrix of type '<class 'numpy.bool_'>'
  	with 17150 stored elements in COOrdinate format>},
 961604: {'data': <2048x2048 sparse matrix of type '<class 'numpy.bool_'>'
  	with 17128 stored e